In [ ]:
import re
import datetime
import pandas

In [ ]:
from IPython.display import display, HTML

In [ ]:
results = pandas.read_csv('~/Downloads/draftkings-contest-entry-history.csv')

In [ ]:
results['year'] = results['Contest_Date_EST'].apply(lambda k: k[0:4])
results['month'] = results['Contest_Date_EST'].apply(lambda k: k[0:7])
results['entry_fee'] = results['Entry_Fee'].apply(lambda k: float(re.sub('\$|,', '', k)))
results['recoup_nonticket'] = results['Winnings_Non_Ticket'].apply(lambda k: float(k.replace('$','').replace(',','')))
results['recoup_ticket']    = results['Winnings_Ticket'].apply(lambda k: float(k.replace('$','').replace(',','')))
results['recoup']           = results['recoup_nonticket'] + results['recoup_ticket']
results['pnl']              = results['recoup'] - results['entry_fee']

In [ ]:
sport_years = results.groupby(['Sport','year','month'], as_index=False).aggregate({'entry_fee': 'sum','recoup': 'sum','pnl':'sum'})
sport_years.sort_values(['pnl',], ascending=False)

In [ ]:
sport_years.groupby(['year', 'month'], as_index=False).aggregate({'pnl':'sum'})

In [ ]:
nba = sport_years[sport_years['Sport']=='NBA']

In [ ]:
nba

In [ ]:
roi_dt = '2016-09'
season_df = nba[nba['month'] > roi_dt]
season_fees = season_df['entry_fee'].sum()
season_recoup = season_df['recoup'].sum()
season_roi = (season_recoup - season_fees) / season_fees
('NBA ROI since {roi_dt}: {season_roi:.0%}'
 .format(roi_dt=roi_dt, season_roi=season_roi))

In [ ]:
nba_contests = results[(results['Sport']=='NBA') & (results['month'] >= roi_dt)]

In [ ]:
nba_slates = (nba_contests.groupby('Contest_Date_EST', as_index=False)
                          .aggregate({'entry_fee': 'sum','recoup': 'sum','pnl':'sum'}))

In [ ]:
nba_slates['pnl'].sum().astype(int)